### **Definir el directorio de trabajo**
Define el directorio de trabajo como la carpeta `data` de la carpeta compartida `DS4A-Team12` de Drive.
Este directorio debe contener el archivo de datos original `Sociodemografico.zip` (5945450 registros, 140 MB) en el directorio `raw_data` y el diccionario de datos `dtypes_sociodemo.json` en el directorio `data_dictionaries`.


In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ICBF/data


In [ ]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np
import json
import re
from utils import replace_log
from utils import impute_nans, impute_years_log, impute_nans_range_log


### **Lectura del archivo**

In [ ]:
time0 = time.time()
# Abre el diccionario con la estructura de datos definida
with open('datatypes_dictionaries/dtypes_vars_toma.json', 'r') as file:
  dtypes_vars_toma = json.load(file)
# Diccionario con categorías válidas (ordenadas) para cada variable categórica (ordinal)
with open('datatypes_dictionaries/tom_cat.json', 'r') as file:
  tom_cat = json.load(file)
# Lee el archivo
tom = pd.read_parquet('clean_data/tomas_normalizado_ids.parquet')
tom = tom.astype(dtypes_vars_toma)
tom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19275081 entries, 0 to 19275080
Data columns (total 26 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   Registro                        UInt32        
 1   Vigencia                        category      
 2   Toma                            category      
 3   Servicio                        category      
 4   FechaValoracionNutricional      datetime64[ns]
 5   EdadMeses                       float32       
 6   FechaMedicionPerimetroBraquial  datetime64[ns]
 7   MedicionPerimetroBraquial       float32       
 8   Peso                            float32       
 9   Talla                           float32       
 10  ZScoreTallaEdad                 float32       
 11  ZScorePesoEdad                  float32       
 12  ZScorePesoTalla                 float32       
 13  ZScoreIMC                       float32       
 14  EstadoTallaEdad                 category      
 

In [ ]:
# Lista de variables categóricas ordinales
cols_cat = [col for col in tom.columns if col in tom_cat.keys()]
# Lista de variables categóricas ordinales
cols_ord = ['Vigencia', 'Toma', 
            'EstadoTallaEdad', 'EstadoPesoEdad', 'EstadoPesoTalla', 'EstadoIMC']
cols_fecha = [col for col in tom.columns if col[:5] == 'Fecha']
cols_medidas = ['MedicionPerimetroBraquial', 'Peso', 'Talla']
cols_zscore = [col for col in tom.columns if col[:6] == 'ZScore']
cols_estado = [col for col in tom.columns if col[:6] == 'Estado']
cols_zscore_peso = [col for col in tom.columns if col[:10] == 'ZScorePeso']
cols_estado_peso = [col for col in tom.columns if col[:10] == 'EstadoPeso']


In [ ]:
# Forzar categorías y ordenar las categorías 
for col in cols_cat:
  categorias = tom_cat[col]
  ordinal = col in cols_ord
  tom[col] = tom[col].cat.set_categories(categorias, ordered = ordinal)

In [ ]:
# Imputar nans
# Para fechas
nan_fechas = {pd.to_datetime('1900 - 1 - 1'),
             pd.to_datetime('1970 - 1 - 1')}
# Para medidas de peso, talla y perímetro braquial
nan_medidas = [0]
for col in cols_fecha:
  old_nans = tom[col].isna().sum()
  tom[col] = tom[col].apply(lambda x: impute_nans(x, nan_fechas))
  new_nans = tom[col].isna().sum() - old_nans
  print(f"En '{col}', {new_nans} NaNs imputados: {time.time()-time0:.2f} s.")

for col in cols_medidas:
  old_nans = tom[col].isna().sum()
  tom[col] = tom[col].apply(lambda x: impute_nans(x, nan_medidas))
  new_nans = tom[col].isna().sum() - old_nans
  print(f"En '{col}', {new_nans} NaNs imputados: {time.time()-time0:.2f} s.")



En 'FechaValoracionNutricional', 0 NaNs imputados: 113.46 s.
En 'FechaMedicionPerimetroBraquial', 11552290 NaNs imputados: 194.25 s.
En 'FechaRegistroSaludNutricion', 0 NaNs imputados: 266.74 s.
En 'MedicionPerimetroBraquial', 11536099 NaNs imputados: 278.37 s.
En 'Peso', 73 NaNs imputados: 288.87 s.
En 'Talla', 216 NaNs imputados: 299.32 s.


In [ ]:
log = []
# Vigencia de los datos
fecha_min, fecha_max = pd.to_datetime('2017 - 1 - 1'), pd.to_datetime('2019 - 12 - 31')
rang_tom = {
  'FechaValoracionNutricional': [fecha_min, fecha_max],
  'EdadMeses': [0, 8 * 12],
  'FechaMedicionPerimetroBraquial': [fecha_min, fecha_max],
  'MedicionPerimetroBraquial': [5, 40],
  'Peso': [0.5, 100],
  'Talla': [20, 200],
  'ZScoreTallaEdad': [-6, 6],
  'ZScorePesoEdad': [-6, 5],
  'ZScorePesoTalla': [-5, 5],
  'ZScoreIMC': [-5, 5],
  }
# Imputar años de vigencia en las fechas de medición que caen fuera de los rangos
out_of_range = tom['FechaMedicionPerimetroBraquial'].\
  apply(lambda x: x < fecha_min or x > fecha_max)

tom.loc[out_of_range, 'FechaMedicionPerimetroBraquial'] = \
  impute_years_log(data=tom.loc[out_of_range], col='FechaMedicionPerimetroBraquial', 
                   col_id='Registro', col_year='Vigencia', log=log)
new_nans = out_of_range.sum()
print(f"En '{col}', {new_nans} NaNs imputados: {time.time()-time0:.2f} s.")

# Reemplazar los valores por fuera de sus rangos naturales por NaNs
for col in rang_tom.keys():
  out_of_range = tom[col].\
    apply(lambda x: x < rang_tom[col][0] or x > rang_tom[col][1])
  tom.loc[out_of_range, col] = \
    impute_nans_range_log(data=tom.loc[out_of_range], col=col, 
                          col_id='Registro', range=rang_tom[col], log=log)
  new_nans = out_of_range.sum()
  print(f"En '{col}', {new_nans} NaNs imputados: {time.time()-time0:.2f} s.")

En 'Talla', 10 NaNs imputados: 354.50 s.
En 'FechaValoracionNutricional', 0 NaNs imputados: 436.72 s.
En 'EdadMeses', 3050 NaNs imputados: 444.87 s.
En 'FechaMedicionPerimetroBraquial', 0 NaNs imputados: 502.28 s.
En 'MedicionPerimetroBraquial', 953 NaNs imputados: 510.37 s.
En 'Peso', 35 NaNs imputados: 518.09 s.
En 'Talla', 9 NaNs imputados: 526.15 s.
En 'ZScoreTallaEdad', 60694 NaNs imputados: 539.73 s.
En 'ZScorePesoEdad', 13583 NaNs imputados: 549.02 s.
En 'ZScorePesoTalla', 45156 NaNs imputados: 561.30 s.
En 'ZScoreIMC', 61448 NaNs imputados: 575.10 s.


In [ ]:
len(log)

184938

In [ ]:
all_zero = tom[cols_zscore].\
  apply(lambda x: list(x) == [0, 0, 0, 0], axis='columns')
tom.loc[all_zero, cols_zscore + cols_estado] = np.nan
new_nans = all_zero.sum()
print(f"""NaNs imputados en columnas de 'ZScore' y 'Estado' en {new_nans} filas con
z-scores = [0, 0, 0, 0]: {time.time() - time0:.2f} s.""")

both_zero = tom[cols_zscore_peso].\
  apply(lambda x: list(x) == [0, 0], axis='columns')
tom.loc[both_zero, cols_zscore_peso + cols_estado_peso] = np.nan
new_nans = both_zero.sum()
print(f"""NaNs imputados en columnas de 'ZScore' y 'Estado' en {new_nans} filas con 
z-scores PesoTalla y PesoEdad = [0, 0]: {time.time() - time0:.2f} s.""")

NaNs imputados en columnas de 'ZScore' y 'Estado' en 23763 filas con
z-scores = [0, 0, 0, 0]: 171.68 s.
NaNs imputados en columnas de 'ZScore' y 'Estado' en 878709 filas con 
z-scores PesoTalla y PesoEdad = [0, 0]: 335.77 s.


In [ ]:
tom = tom.sort_values(['IdBeneficiario', 'FechaValoracionNutricional', 'Toma'])
tom = tom.reset_index(drop=True).reset_index().rename(columns={'index': 'IdToma'})
tom.head()

,IdToma,Registro,Vigencia,Toma,Servicio,FechaValoracionNutricional,EdadMeses,FechaMedicionPerimetroBraquial,MedicionPerimetroBraquial,Peso,Talla,ZScoreTallaEdad,ZScorePesoEdad,ZScorePesoTalla,ZScoreIMC,EstadoTallaEdad,EstadoPesoEdad,EstadoPesoTalla,EstadoIMC,Flag,FechaRegistroSaludNutricion,PresentaCarneVacunacion,ControlesCrecimDesarrollo,AntecedentePremadurez,Direccion,IdBeneficiario,Id
0,0,988318,2017,1,HCB TRADICIONAL- COMUNITARIO (T),2017-02-09,48.0,NaT,NaN,17.0,98.000000,-1.29,0.29,1.65,1.70,Riesgo de baja talla,Peso adecuado para la edad,Riesgo de sobrepeso,Riesgo de sobrepeso,0,NaT,S,0,NaN,Primera Infancia,36,<NA>
1,1,988318,2017,2,HCB TRADICIONAL- COMUNITARIO (T),2017-05-05,50.0,NaT,NaN,18.0,104.000000,-0.23,0.51,0.99,1.00,Talla adecuada para la edad,Peso adecuado para la edad,Peso adecuado para la talla,Adecuado para la edad,0,NaT,S,0,NaN,Primera Infancia,36,<NA>
2,2,988318,2017,3,HCB TRADICIONAL- COMUNITARIO (T),2017-08-01,53.0,NaT,NaN,16.0,98.699997,-1.79,-0.61,0.78,0.86,Riesgo de baja talla,Peso adecuado para la edad,Peso adecuado para la talla,Adecuado para la edad,0,NaT,S,1,NaN,Primera Infancia,36,<NA>
3,3,988318,2017,4,HCB TRADICIONAL- COMUNITARIO (T),2017-11-11,57.0,NaT,NaN,16.0,98.699997,-2.15,-0.84,0.78,0.87,Retraso en talla,Peso adecuado para la edad,Peso adecuado para la talla,Adecuado para la edad,0,NaT,S,1,NaN,Primera Infancia,36,<NA>
4,4,988319,2017,3,HCB TRADICIONAL- COMUNITARIO (T),2017-08-01,52.0,NaT,NaN,16.0,108.000000,0.55,-0.48,-1.27,-1.31,Talla adecuada para la edad,Peso adecuado para la edad,Riesgo de desnutrición aguda,Riesgo para la delgadez,0,NaT,S,1,NaN,Primera Infancia,529,2028706


In [ ]:
tom['IdToma'] = tom['IdToma'].astype('UInt32')
tom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19275081 entries, 0 to 19275080
Data columns (total 27 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   IdToma                          UInt32        
 1   Registro                        UInt32        
 2   Vigencia                        category      
 3   Toma                            category      
 4   Servicio                        category      
 5   FechaValoracionNutricional      datetime64[ns]
 6   EdadMeses                       float32       
 7   FechaMedicionPerimetroBraquial  datetime64[ns]
 8   MedicionPerimetroBraquial       float64       
 9   Peso                            float64       
 10  Talla                           float64       
 11  ZScoreTallaEdad                 float32       
 12  ZScorePesoEdad                  float32       
 13  ZScorePesoTalla                 float32       
 14  ZScoreIMC                       float32       
 

In [ ]:
tom.to_parquet('clean_data/tomas.parquet')

In [ ]:
tom[tom.IdBeneficiario.isna()]

,IdToma,Registro,Vigencia,Toma,Servicio,FechaValoracionNutricional,EdadMeses,FechaMedicionPerimetroBraquial,MedicionPerimetroBraquial,Peso,Talla,ZScoreTallaEdad,ZScorePesoEdad,ZScorePesoTalla,ZScoreIMC,EstadoTallaEdad,EstadoPesoEdad,EstadoPesoTalla,EstadoIMC,Flag,FechaRegistroSaludNutricion,PresentaCarneVacunacion,ControlesCrecimDesarrollo,AntecedentePremadurez,Direccion,IdBeneficiario,Id


In [ ]:
primera_toma = [0]
for i in range(1, len(tom[1:])):
  if tom.loc[i, 'IdBeneficiario'] != tom.loc[i - 1, 'IdBeneficiario']:
    primera_toma.append(i)
  if i % 1e6 == 0:
    print(f'Toma {i}: {time.time() - time0:.2f} s.')
primera_toma_beneficiario = \
  pd.DataFrame({'IdBeneficiario': tom['IdBeneficiario'].unique(),
                'IdToma': primera_toma})
print(f'Primera toma por beneficiario: {time.time() - time0:.2f} s.')


Toma 1000000: 40.09 s.
Toma 2000000: 78.90 s.
Toma 3000000: 117.98 s.
Toma 4000000: 157.15 s.
Toma 5000000: 196.42 s.
Toma 6000000: 235.38 s.
Toma 7000000: 274.52 s.
Toma 8000000: 313.97 s.
Toma 9000000: 353.58 s.
Toma 10000000: 393.20 s.
Toma 11000000: 432.09 s.
Toma 12000000: 471.54 s.
Toma 13000000: 510.85 s.
Toma 14000000: 550.61 s.
Toma 15000000: 590.06 s.
Toma 16000000: 629.44 s.
Toma 17000000: 669.18 s.
Toma 18000000: 708.92 s.
Toma 19000000: 749.07 s.
Primera toma por beneficiario: 765.30 s.


In [ ]:
primera_toma = primer_IdToma

In [ ]:
print(f'Longitud lista: {len(primera_toma_beneficiario)}')
primera_toma[-3:]

Longitud lista: 3362645


[19275074, 19275076, 19275078]

In [ ]:
primera_toma_beneficiario = \
  pd.DataFrame({'IdBeneficiario': tom['IdBeneficiario'].unique(),
                'IdToma': primera_toma})
primera_toma_beneficiario.to_parquet('auxiliary_data/beneficiario_primera_toma.parquet')
primera_toma_beneficiario

,IdBeneficiario,IdToma
0,36,0
1,529,4
2,8872,5
3,10058,13
4,15997,15
...,...,...
3362640,18973818,19275071
3362641,18973819,19275073
3362642,18973820,19275074
3362643,18973821,19275076


In [ ]:
for i in range(10):
  print(list(range(primera_toma[i], primera_toma[i + 1])))

[0, 1, 2, 3]
[4]
[5, 6, 7, 8, 9, 10, 11, 12]
[13, 14]
[15, 16, 17]
[18, 19, 20, 21]
[22, 23]
[24]
[25, 26, 27, 28, 29, 30, 31]
[32, 33]


In [ ]:
puntos_corte =  {
  'TallaEdad': [-2, -1, 3],
  'PesoTalla': [-3, -2, -1, 1, 2, 3],
  'PesoEdad': [-3, -2, -1, 1, 2],
  'IMC': [-2, -1, 1, 2, 3]
  }
var = 'TallaEdad'
puntos_corte[var]
categories = tom[f'Estado{var}'].cat.categories
x = 51.0

def nutritional_state(z_score, cuttoff_values, categories):
  if z_score not in cuttoff_values:
    return categories[sorted(cuttoff_values + [z_score]).index(z_score)]
  else:
    new_zcore = z_score + 0.5
    return categories[sorted(cuttoff_values + [new_zcore]).index(new_zcore)]
t0 = time.time()


for var in puntos_corte.keys():
  categorias = tom[f'Estado{var}'].cat.categories
  w = tom[f'ZScore{var}'].\
    apply(lambda x: nutritional_state(x, puntos_corte[var], categorias))
  print(var)
  print((time.time() - t0))
w.head()

TallaEdad
37.43217611312866
PesoTalla
80.32051277160645
PesoEdad
121.61056756973267
IMC
162.04035449028015


0        Riesgo de sobrepeso
1        Riesgo de sobrepeso
2      Adecuado para la edad
3      Adecuado para la edad
4    Riesgo para la delgadez
Name: ZScoreIMC, dtype: object